# Enviroweather Private Weather Stations Python package: `ewx_pws`
## Example usage

Using this package in a project to pull weather data you start as follows

In [1]:
import json
from pprint import pprint
from ewx_pws import __version__, ewx_pws
print(__version__)

0.1.0


This package works with the following station types:

In [2]:
print(ewx_pws.STATION_TYPE_LIST)

['ZENTRA', 'ONSET', 'DAVIS', 'RAINWISE', 'SPECTRUM', 'LOCOMOS', 'GENERIC']


The main function accepts configuration information for one or more weather stations API (clouds), 
and uses that to connect to the API for that station and pull data for a specific time interval. 

The configuration can be saved in a CSV file or in the future in a database table. 

This example usage notebook assumes the values are in a CSV file named 'test_stations.csv'  Alter the variable for this file name
to use a different file. 

The file must be in the format :  


In [3]:
# this file must be present for the rest of the notebook to work.  You can create this file with any name, 
# but must be loaded here
station_config_file = "../test_stations.csv"

In [4]:
# load weather station configuration from an environment file
stations = ewx_pws.station_dict_from_file(station_config_file)

print("full station config:")
for station in stations.values():
    print(station.config)

2023-09-12 16:10:28,653-11628-DEBUG-instantiating EWX-METER-FO
2023-09-12 16:10:28,654-11628-DEBUG-instantiating EWXDAVIS01
2023-09-12 16:10:28,654-11628-DEBUG-instantiating EWXSPECTRUM01
2023-09-12 16:10:28,655-11628-DEBUG-instantiating EWXONSET01
2023-09-12 16:10:28,655-11628-DEBUG-instantiating EWXRAINWISE01
2023-09-12 16:10:28,656-11628-DEBUG-instantiating EWXLOCOMOS06
2023-09-12 16:10:28,656-11628-DEBUG-instantiating EWX-METER-FO2


full station config:
station_id='EWX-METER-FO' install_date=datetime.datetime(2023, 5, 1, 0, 0) station_type='ZENTRA' tz='ET' sn='z6-12564' token='5b8f637a718b9e3ad6e31eec81f0b51d91ac38bd'
station_id='EWXDAVIS01' install_date=datetime.datetime(2023, 5, 1, 0, 0) station_type='DAVIS' tz='ET' sn='117005' apikey='aoqdcbirudd1sarq6erfj6tgtw67sbvo' apisec='elvr77yhdx3p0286cwnnqqstdwqg8zxf'
station_id='EWXSPECTRUM01' install_date=datetime.datetime(2023, 5, 1, 0, 0) station_type='SPECTRUM' tz='ET' sn='50400123' apikey='11a5c3a939856b08677b7a072f8e6865'
station_id='EWXONSET01' install_date=datetime.datetime(2023, 5, 1, 0, 0) station_type='ONSET' tz='ET' sn='21092695' client_id='Enviroweather_WS' client_secret='75d2b7f58f5d0cac699f5b9616318be268057de6' ret_form='JSON' user_id='12848' sensor_sn={'atemp': '21079936-1', 'pcpn': '21085496-1', 'relh': '21079936-2'}
station_id='EWXRAINWISE01' install_date=datetime.datetime(2023, 5, 1, 0, 0) station_type='RAINWISE' tz='ET' username='200000000500' sid='

In [5]:
print("stations in config file:")
for station in stations.values():
    print('----------')
    print('id:      ', station.id)
    print('type:    ', station.config.station_type)
    print('timezone:', station.config.tz)
    print('sample interval:', station.interval_min)



stations in config file:
----------
id:       EWX-METER-FO
type:     ZENTRA
timezone: ET
sample interval: 5
----------
id:       EWXDAVIS01
type:     DAVIS
timezone: ET
sample interval: 15
----------
id:       EWXSPECTRUM01
type:     SPECTRUM
timezone: ET
sample interval: 5
----------
id:       EWXONSET01
type:     ONSET
timezone: ET
sample interval: 5
----------
id:       EWXRAINWISE01
type:     RAINWISE
timezone: ET
sample interval: 15
----------
id:       EWXLOCOMOS06
type:     LOCOMOS
timezone: ET
sample interval: 30
----------
id:       EWX-METER-FO2
type:     ZENTRA
timezone: ET
sample interval: 5


Get sample weather specifying a time interval.,  UTCInterval is a class that can create intervals of recent times, and ensure times created are UTC so The station classes only accept timestamps that have a timezone, and that timezone must be UTC.   

In [6]:
# set the time 
from ewx_pws.time_intervals import UTCInterval
interval = UTCInterval.previous_interval(delta_mins = 60)
interval.dict()

{'start': datetime.datetime(2023, 9, 12, 19, 0, tzinfo=datetime.timezone.utc),
 'end': datetime.datetime(2023, 9, 12, 20, 0, tzinfo=datetime.timezone.utc)}

In [7]:
# get weather one station from the list.  test by changing the station_id with one from above
station_id = 'EWX-METER-FO2'
station = stations[station_id]

pprint(station.config)

ZentraConfig(station_id='EWX-METER-FO2', install_date=datetime.datetime(2023, 5, 1, 0, 0), station_type='ZENTRA', tz='ET', sn='z6-20553', token='12cc21d9f697e478885cca9b65307d27872dfb18')


The output uses a class WeatherReading which has metadata, and then a list of responses from the API.  Some APIs require multiple calls to the API for periods > 24 hours, so the responses are always stored in a list

In [8]:
api_data = station.get_readings(interval.start, interval.end)
pprint(api_data)


2023-09-12 16:10:37,650-11628-DEBUG-Starting new HTTPS connection (1): zentracloud.com:443
2023-09-12 16:10:43,481-11628-DEBUG-https://zentracloud.com:443 "GET /api/v4/get_readings/?device_sn=z6-20553&start_date=2023-09-12+15%3A00%3A00&end_date=2023-09-12+16%3A00%3A00&page_num=1&per_page=1000 HTTP/1.1" 200 None


WeatherAPIData(station_id='EWX-METER-FO2', station_type='ZENTRA', request_id='ca23cfa5-b418-4a1f-9a99-ef42441bebba', request_datetime=datetime.datetime(2023, 9, 13, 0, 10, 37, 640985, tzinfo=datetime.timezone.utc), time_interval=UTCInterval(start=datetime.datetime(2023, 9, 12, 19, 0, tzinfo=datetime.timezone.utc), end=datetime.datetime(2023, 9, 12, 20, 0, tzinfo=datetime.timezone.utc)), package_version='0.1', responses=[WeatherAPIResponse(url='https://zentracloud.com/api/v4/get_readings/?device_sn=z6-20553&start_date=2023-09-12+15%3A00%3A00&end_date=2023-09-12+16%3A00%3A00&page_num=1&per_page=1000', status_code='200', reason='OK', text='{"pagination": {"per_page": 1000, "page_num": 1, "next_url": "https://zentracloud.com/api/v4/get_readings/?device_sn=z6-20553&start_date=2023-09-12+15%3A00%3A00&end_date=2023-09-12+16%3A00%3A00&page_num=2&per_page=1000", "max_mrid": 44168, "page_num_readings": 13, "page_num_outputs": 312, "page_start_date": "2023-09-12 15:00:00-04:00", "page_end_date": 

## raw api output

let's look at what comes out of the station API for our example station.  The actual data from the api is in the responses property of the weather_data object.  This is an array as some stations return an array of responses (Davis only allows maximum 24hrs per reading)

The folllowing looks at just the first response in the array

In [9]:
for response in  api_data.responses:
    pprint(response.text)

('{"pagination": {"per_page": 1000, "page_num": 1, "next_url": '
 '"https://zentracloud.com/api/v4/get_readings/?device_sn=z6-20553&start_date=2023-09-12+15%3A00%3A00&end_date=2023-09-12+16%3A00%3A00&page_num=2&per_page=1000", '
 '"max_mrid": 44168, "page_num_readings": 13, "page_num_outputs": 312, '
 '"page_start_date": "2023-09-12 15:00:00-04:00", "page_end_date": "2023-09-12 '
 '16:00:00-04:00", "page_start_mrid": 44153, "page_end_mrid": 44168}, "data": '
 '{"Air Temperature": [{"metadata": {"device_sn": "z6-20553", "device_name": '
 '"envwx-meter-fo2", "port_number": 1, "sensor_sn": "ATM-410008286", '
 '"sensor_name": "ATMOS 41", "units": " \\u00b0C"}, "readings": '
 '[{"timestamp_utc": 1694548800, "datetime": "2023-09-12 16:00:00-04:00", '
 '"tz_offset": -14400, "value": 17.5, "precision": 1, "mrid": 44168, '
 '"error_flag": false, "error_description": null}, {"timestamp_utc": '
 '1694548500, "datetime": "2023-09-12 15:55:00-04:00", "tz_offset": -14400, '
 '"value": 17.2, "precisi

The output from the api is a mix of the JSON output from the stations and metadata about how the request for data was made.   Each station has it's own format from the API, and we want to transform that to a standardize format for EWX. 

The station object stores the most recent request, and can run the `transform` method without an argument, and it will to use the latest api request, or you can send data. 



In [11]:
readings = station.transform(api_data)
for reading in readings:
    pprint(reading)

2023-09-12 16:12:12,426-11628-DEBUG-Zentra readings found
2023-09-12 16:12:12,427-11628-DEBUG-transformed_reading type <class 'dict_values'>: dict_values([{'data_datetime': datetime.datetime(2023, 9, 12, 20, 0, tzinfo=datetime.timezone.utc), 'atemp': 17.5, 'lws0': 0.0, 'pcpn': 0.0, 'relh': 83.3}, {'data_datetime': datetime.datetime(2023, 9, 12, 19, 55, tzinfo=datetime.timezone.utc), 'atemp': 17.2, 'lws0': 1.0, 'pcpn': 0.0, 'relh': 83.6}, {'data_datetime': datetime.datetime(2023, 9, 12, 19, 50, tzinfo=datetime.timezone.utc), 'atemp': 17.3, 'lws0': 5.0, 'pcpn': 0.017, 'relh': 83.9}, {'data_datetime': datetime.datetime(2023, 9, 12, 19, 45, tzinfo=datetime.timezone.utc), 'atemp': 17.8, 'lws0': 5.0, 'pcpn': 0.0, 'relh': 81.5}, {'data_datetime': datetime.datetime(2023, 9, 12, 19, 40, tzinfo=datetime.timezone.utc), 'atemp': 19.6, 'lws0': 1.0, 'pcpn': 0.0, 'relh': 74.7}, {'data_datetime': datetime.datetime(2023, 9, 12, 19, 35, tzinfo=datetime.timezone.utc), 'atemp': 20.4, 'lws0': 0.0, 'pcpn': 

{'pagination': {'per_page': 1000, 'page_num': 1, 'next_url': 'https://zentracloud.com/api/v4/get_readings/?device_sn=z6-20553&start_date=2023-09-12+15%3A00%3A00&end_date=2023-09-12+16%3A00%3A00&page_num=2&per_page=1000', 'max_mrid': 44168, 'page_num_readings': 13, 'page_num_outputs': 312, 'page_start_date': '2023-09-12 15:00:00-04:00', 'page_end_date': '2023-09-12 16:00:00-04:00', 'page_start_mrid': 44153, 'page_end_mrid': 44168}, 'data': {'Air Temperature': [{'metadata': {'device_sn': 'z6-20553', 'device_name': 'envwx-meter-fo2', 'port_number': 1, 'sensor_sn': 'ATM-410008286', 'sensor_name': 'ATMOS 41', 'units': ' °C'}, 'readings': [{'timestamp_utc': 1694548800, 'datetime': '2023-09-12 16:00:00-04:00', 'tz_offset': -14400, 'value': 17.5, 'precision': 1, 'mrid': 44168, 'error_flag': False, 'error_description': None}, {'timestamp_utc': 1694548500, 'datetime': '2023-09-12 15:55:00-04:00', 'tz_offset': -14400, 'value': 17.2, 'precision': 1, 'mrid': 44167, 'error_flag': False, 'error_descr

## Using the data

if Pandas is installed, can read that transformed data into a table as follows

In [14]:
import pandas
from IPython.display import display
weather_df = pandas.DataFrame(readings.for_csv())
display(weather_df)

,station_id,station_type,request_id,request_datetime,time_interval,data_datetime,atemp,pcpn,relh,lws0
0,EWX-METER-FO2,ZENTRA,ca23cfa5-b418-4a1f-9a99-ef42441bebba,2023-09-13 00:10:37.640985+00:00,"{'start': 2023-09-12 19:00:00+00:00, 'end': 20...",2023-09-12 20:00:00+00:00,17.5,0.000,83.3,0.0
1,EWX-METER-FO2,ZENTRA,ca23cfa5-b418-4a1f-9a99-ef42441bebba,2023-09-13 00:10:37.640985+00:00,"{'start': 2023-09-12 19:00:00+00:00, 'end': 20...",2023-09-12 19:55:00+00:00,17.2,0.000,83.6,1.0
2,EWX-METER-FO2,ZENTRA,ca23cfa5-b418-4a1f-9a99-ef42441bebba,2023-09-13 00:10:37.640985+00:00,"{'start': 2023-09-12 19:00:00+00:00, 'end': 20...",2023-09-12 19:50:00+00:00,17.3,0.017,83.9,5.0
3,EWX-METER-FO2,ZENTRA,ca23cfa5-b418-4a1f-9a99-ef42441bebba,2023-09-13 00:10:37.640985+00:00,"{'start': 2023-09-12 19:00:00+00:00, 'end': 20...",2023-09-12 19:45:00+00:00,17.8,0.000,81.5,5.0
4,EWX-METER-FO2,ZENTRA,ca23cfa5-b418-4a1f-9a99-ef42441bebba,2023-09-13 00:10:37.640985+00:00,"{'start': 2023-09-12 19:00:00+00:00, 'end': 20...",2023-09-12 19:40:00+00:00,19.6,0.000,74.7,1.0
5,EWX-METER-FO2,ZENTRA,ca23cfa5-b418-4a1f-9a99-ef42441bebba,2023-09-13 00:10:37.640985+00:00,"{'start': 2023-09-12 19:00:00+00:00, 'end': 20...",2023-09-12 19:35:00+00:00,20.4,0.000,71.7,0.0
6,EWX-METER-FO2,ZENTRA,ca23cfa5-b418-4a1f-9a99-ef42441bebba,2023-09-13 00:10:37.640985+00:00,"{'start': 2023-09-12 19:00:00+00:00, 'end': 20...",2023-09-12 19:30:00+00:00,20.4,0.000,71.7,0.0
7,EWX-METER-FO2,ZENTRA,ca23cfa5-b418-4a1f-9a99-ef42441bebba,2023-09-13 00:10:37.640985+00:00,"{'start': 2023-09-12 19:00:00+00:00, 'end': 20...",2023-09-12 19:25:00+00:00,20.5,0.000,71.9,0.0
8,EWX-METER-FO2,ZENTRA,ca23cfa5-b418-4a1f-9a99-ef42441bebba,2023-09-13 00:10:37.640985+00:00,"{'start': 2023-09-12 19:00:00+00:00, 'end': 20...",2023-09-12 19:20:00+00:00,20.2,0.000,72.6,0.0
9,EWX-METER-FO2,ZENTRA,ca23cfa5-b418-4a1f-9a99-ef42441bebba,2023-09-13 00:10:37.640985+00:00,"{'start': 2023-09-12 19:00:00+00:00, 'end': 20...",2023-09-12 19:15:00+00:00,20.1,0.000,72.6,0.0
